In [14]:
# 加载预训练特征提取模型
# reference website: https://wikipedia2vec.github.io/wikipedia2vec/usage/#api-usage
import numpy as np
from wikipedia2vec import Wikipedia2Vec
# 必须下载二进制文件否则下一步会报错，浪费我一个小时，怒😡
wiki2vec = Wikipedia2Vec.load('zhwiki_20180420_100d.pkl')
# test
vec = wiki2vec.get_word_vector('我')
vectorsize = vec.size
print(vec,'\n',vectorsize)

[-0.15454428  0.52875054 -0.47713432  0.26700583  0.5780986  -0.2584704
 -0.497395   -0.37840477 -0.04801681 -0.1354587  -0.30140558  0.3864191
  0.10743058 -0.16617118 -0.04376784  0.19094634 -0.662355   -0.09070095
 -0.23156993 -0.5459718  -0.10523433 -0.20469089  0.09074577 -0.08418999
 -0.6287511  -0.4343539  -0.17811942 -0.26209912  0.59038424 -0.03774031
 -0.14668883  0.30349293  0.03785542 -0.2719773  -0.32126015 -0.06889755
  0.03629145  0.33375648  0.01039888 -0.12757844  0.16452992 -0.07805916
  0.6332627  -0.00853407  0.5796044  -0.4813189  -0.415879   -0.18914485
  0.37880462 -0.535376    0.27505007 -0.0053158   0.43196863 -0.10982956
 -0.2483539   0.57755494  1.2750691  -0.2753227   0.21096511  0.17983645
  0.07333843  0.32510185  0.21086293  0.35948765 -1.0670815  -0.24465425
 -0.05507161  0.22329137  0.18268651  0.23923065 -0.15428372  0.45899114
 -0.07298146 -0.14948614  0.28234842  0.2845556   0.48830715  0.0445042
  0.2580861   0.5829236  -0.24267407 -0.1792513   0.06

In [19]:
import re
#定义删除除字母,数字，汉字以外的所有符号的函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line

#停用词列表
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

#加载停用词
stopwords = stopwordslist("./stopwords.txt")

import pandas as pd
# df = pd.read_csv('./data_lite_all.csv')
df = pd.read_csv('./data_cleaned_1.csv')
df=df[['text','label']]
print("数据总量: %d ." % len(df))
df.sample(10)

数据总量: 2000 .


,text,label
325,谢谢，是我孤陋寡闻，洛必达法则没用过,0
1375,函数的求导法则请老师写下图片中红框部分的演算步骤，以及下边蓝色划线部分怎么来的,2
1692,求老师解答,3
298,x→∞，是x趋向于正无穷，还是趋向于无穷,0
1171,请问第二题怎么做啊,2
1440,dy代表什么,2
1283,老师，为什么极限为啊,2
1621,哪两个,3
1735,能不能写一下式子鸭,3
1699,老师请问这题该怎么解谢谢,3


In [20]:
import jieba as jb
#删除除字母,数字，汉字以外的所有符号
df['clean_text'] = df['text'].apply(remove_punctuation)

#分词，并过滤停用词
# df['cut_review'] = df['clean_review'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df['cut_text'] = df['clean_text'].apply(lambda x: [w for w in list(jb.cut(x)) if w not in stopwords])
df.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\WINDOWS\TEMP\jieba.cache
Loading model cost 0.489 seconds.
Prefix dict has been built successfully.


,text,label,clean_text,cut_text
0,为什么无界函数不一定为无穷大,0,为什么无界函数不一定为无穷大,"[为什么, 无, 界, 函数, 不, 一定, 为, 无穷大]"
1,判断，处有无定义，若，处有定义，但是，不存在，即，不存在或，不存在，若，处有定义存在，但是,0,判断处有无定义若处有定义但是不存在即不存在或不存在若处有定义存在但是,"[判断, 处, 有无, 定义, 若处, 有, 定义, 但是, 不, 存在, 即, 不, 存在..."
2,谢谢,0,谢谢,[谢谢]
3,先开上面三次方再求导,0,先开上面三次方再求导,"[先开, 上面, 三次方, 再, 求导]"
4,求解答这里是讲收敛数列的有界性，为什么举例是这个例子这个数列并不收敛呀这个数列不是发散的吗,0,求解答这里是讲收敛数列的有界性为什么举例是这个例子这个数列并不收敛呀这个数列不是发散的吗,"[求, 解答, 这里, 是, 讲, 收敛, 数列, 的, 有界性, 为什么, 举例, 是, ..."


In [21]:
from gensim.models.doc2vec import TaggedDocument
#创建标签化文档
train_tagged = df.apply(
    lambda r: TaggedDocument(words=r['cut_text'], tags=[r['label']]), axis=1)
train_tagged.sample(10)

79                                     ([还有, 种, 方法], [0])
344     ([函数, 的, 连续性, 麻烦, 老师, 看下例, 和, 例中, 画圈, 的, 部分, 怎...
309            ([老师, 求解, 下面, 几个, 题目, 请, 老师, 解答, 一下], [0])
1973                             ([能, 帮忙, 解, 一下, 吗], [3])
1328                        ([请问, 这个, 极限, 怎么, 求, 啊], [2])
1869    ([想要, 知道, 详细, 的, 过程, 和, 每, 一步, 的, 依据, 如图, 希望, ...
1200                            ([老师, 这题, 怎么, 做, 啊], [2])
1565                                   ([谁, 能, 帮帮我], [3])
59                                      ([啊, 我会, 了], [0])
1758                              ([老师, 这道题, 怎么, 做], [3])
dtype: object

In [44]:
text1=train_tagged[0][0]
text2=train_tagged[0][1]
text1[1],text2,text2[0]

('无', [0], 0)

In [48]:
import numpy as np
import torch
# 分词后查阅预训练模型中每个单词的特征，然后取平均值，为避免词典中没有某个词而导致查询时整个程序停止，引入系统中
# 初始化数据
data_length = 2000
word_vector_size = 100
label = torch.zeros(data_length)
features = torch.zeros(data_length,word_vector_size)
for i in range(len(train_tagged)):
    cut_text = train_tagged[i][0]  # 分词后文本
    label[i] = train_tagged[i][1][0]  # 标签存入label
    # 每个词的词向量
    count = 0
    for j in range(len(cut_text)):
        try:
            vec_temp = wiki2vec.get_word_vector(cut_text[j]) # 获取当前词的词向量
            features[i] += vec_temp
            # print(vec_temp)
            count += 1
        except Exception as f:
            # print("Exception f")
            ex = 1
    if count!=0: features[i] /= count
    # print(features[i])
    # break
features[1999],label

(tensor([-0.2924,  0.1533, -0.1521,  0.4695,  0.1799,  0.1175, -0.1602, -0.7899,
          0.7427, -0.0958,  0.0947,  0.2020, -0.4102, -0.1616,  0.1495, -0.2397,
         -0.2724,  0.0600, -0.0103, -0.1817, -0.1020,  0.0334,  0.2580, -0.1205,
         -0.5100, -0.3163, -0.0271, -0.1161,  0.2077, -0.1881,  0.2675,  0.2193,
          0.5156,  0.3702, -0.0327, -0.1584,  0.3373, -0.1080, -0.0819, -0.1874,
          0.1980,  0.1727,  0.3929, -0.3468,  0.5301, -0.4854, -0.5363, -0.2199,
          0.1926, -0.6094,  0.0602,  0.6900,  0.4246,  0.4248,  0.2148, -0.0471,
          0.8418, -0.2958,  0.3884,  0.5980,  0.3890,  0.3865, -0.3905, -0.1531,
         -0.7449, -0.0657,  0.0027,  0.4643,  0.1590, -0.1030,  0.1285,  0.5606,
          0.3592, -0.1796, -0.2919,  0.0113,  0.3375, -0.1267,  0.2550,  0.5555,
         -0.1158, -0.1533, -0.0063, -0.5107, -0.0514, -0.2981, -0.2564,  0.0921,
         -0.1424, -0.5418,  0.2905, -0.2002, -0.1603,  0.9346,  0.0510,  0.0556,
          0.5027,  0.0431, -

In [73]:
# 后端网络LPA分类
import numpy as np
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import classification_report, accuracy_score

X = features  # 输入
y = label  # 输出
rng = np.random.RandomState(9)  # 随机数种子
indices = np.arange(len(y))
rng.shuffle(indices)  # 将索引打乱
# print(X[0])

n_total_samples = len(y)
# n_labeled_points = 2795  # 已标记/样本数
n_labeled_points = 1600  # 已标记样本数1260 1440

unlabeled_set = indices[n_labeled_points:]

y_train = np.copy(y)
y_train[unlabeled_set] = -1  # 未标记样本集y设为-1

num=100
accuracy = []
gammas = np.logspace(-2, 2, num=num)
# gammas = range(100)
score_in = 0
for i in range(num):
    lp_model = LabelSpreading(gamma=gammas[i], max_iter=30, kernel='rbf')
    lp_model.fit(X, y_train)
    predicted_labels = lp_model.transduction_[unlabeled_set]
    true_labels = y[unlabeled_set]
    score = accuracy_score(true_labels, predicted_labels)
    # if score > score_in:
    #     gamma_fin = gammas[i]
    accuracy.append(score)
    print(i,gammas[i],score)
    print(classification_report(true_labels, predicted_labels, digits=4))

# lp_model = LabelSpreading(gamma=0.24, max_iter=200, kernel='rbf')# bert2000
lp_model = LabelSpreading(gamma=4.75, max_iter=200, kernel='rbf')# tf-idf2000
# lp_model = LabelSpreading(kernel='knn', n_neighbors=9, max_iter=100)
lp_model.fit(X, y_train)
predicted_labels = lp_model.transduction_[unlabeled_set]
true_labels = y[unlabeled_set]
print(classification_report(true_labels, predicted_labels, digits=4))



# print(
#     "Label Spreading model: %d labeled & %d unlabeled points (%d total)"
#     % (n_labeled_points, n_total_samples - n_labeled_points, n_total_samples)
# )
# print(classification_report(true_labels, predicted_labels))
# print(true_labels)
# print(predicted_labels)


0 0.01 0.3625
              precision    recall  f1-score   support

         0.0     0.9231    0.2400    0.3810       100
         1.0     0.7027    0.2549    0.3741       102
         2.0     0.2819    0.9694    0.4368        98
         3.0     0.0000    0.0000    0.0000       100

    accuracy                         0.3625       400
   macro avg     0.4769    0.3661    0.2980       400
weighted avg     0.4790    0.3625    0.2976       400

1 0.010974987654930561 0.3725
              precision    recall  f1-score   support

         0.0     0.9259    0.2500    0.3937       100
         1.0     0.6829    0.2745    0.3916       102
         2.0     0.2857    0.9592    0.4403        98
         3.0     0.6667    0.0200    0.0388       100

    accuracy                         0.3725       400
   macro avg     0.6403    0.3759    0.3161       400
weighted avg     0.6423    0.3725    0.3159       400



D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2 0.012045035402587823 0.3875
              precision    recall  f1-score   support

         0.0     0.9259    0.2500    0.3937       100
         1.0     0.6939    0.3333    0.4503       102
         2.0     0.2928    0.9592    0.4487        98
         3.0     0.6667    0.0200    0.0388       100

    accuracy                         0.3875       400
   macro avg     0.6448    0.3906    0.3329       400
weighted avg     0.6468    0.3875    0.3329       400

3 0.013219411484660288 0.395
              precision    recall  f1-score   support

         0.0     0.9286    0.2600    0.4062       100
         1.0     0.6981    0.3627    0.4774       102
         2.0     0.2943    0.9490    0.4493        98
         3.0     0.6667    0.0200    0.0388       100

    accuracy                         0.3950       400
   macro avg     0.6469    0.3979    0.3429       400
weighted avg     0.6489    0.3950    0.3431       400

4 0.014508287784959394 0.4025
              precision    recall  f1-sco

In [60]:
lp_model = LabelSpreading(gamma=7.4, max_iter=200, kernel='rbf')# tf-idf2000
# lp_model = LabelSpreading(kernel='knn', n_neighbors=9, max_iter=100)
lp_model.fit(X, y_train)
predicted_labels = lp_model.transduction_[unlabeled_set]
true_labels = y[unlabeled_set]
print(classification_report(true_labels, predicted_labels, digits=4))

              precision    recall  f1-score   support

         0.0     0.6348    0.7374    0.6822        99
         1.0     0.7576    0.6637    0.7075       113
         2.0     0.5146    0.5300    0.5222       100
         3.0     0.6506    0.6136    0.6316        88

    accuracy                         0.6375       400
   macro avg     0.6394    0.6362    0.6359       400
weighted avg     0.6429    0.6375    0.6382       400



In [130]:
# RF
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

X = features  # 输入
y = label  # 输出

rng = np.random.RandomState(9)  # 随机数种子
indices = np.arange(len(y))
rng.shuffle(indices)  # 将索引打乱
# print(X[0])

n_total_samples = len(y)
n_labeled_points = 1600  # 已标记样本数

unlabeled_set = indices[n_labeled_points:]
labeled_set = indices[:n_labeled_points]
X_train = X[labeled_set]
y_train = y[labeled_set]
X_test = X[unlabeled_set]
y_true = y[unlabeled_set]

model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(classification_report(y_true, y_predict, digits=4))

# # 设置弱学习器数量为10
# for i in range(1,100):
#     model = RandomForestClassifier(n_estimators=i)
#     model.fit(X_train, y_train)
#     y_predict = model.predict(X_test)
#     print(i, accuracy_score(y_true, y_predict))

              precision    recall  f1-score   support

         0.0     0.6486    0.7200    0.6825       100
         1.0     0.7412    0.6176    0.6738       102
         2.0     0.4318    0.3878    0.4086        98
         3.0     0.5862    0.6800    0.6296       100

    accuracy                         0.6025       400
   macro avg     0.6020    0.6014    0.5986       400
weighted avg     0.6035    0.6025    0.5999       400



In [63]:
model = RandomForestClassifier(n_estimators=39)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(classification_report(y_true, y_predict, digits=4))

              precision    recall  f1-score   support

         0.0     0.6854    0.6162    0.6489        99
         1.0     0.6522    0.7965    0.7171       113
         2.0     0.5000    0.4600    0.4792       100
         3.0     0.6543    0.6023    0.6272        88

    accuracy                         0.6250       400
   macro avg     0.6230    0.6187    0.6181       400
weighted avg     0.6228    0.6250    0.6210       400

